In [3]:
import pandas as pd
titanic = pd.read_csv("data/titanic.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# ¿Cómo calcular las estadísticas de resumen?

## 1. Agregando estadísticas

<img src="data/06_aggregate.svg">

### 1.1 ¿Cuál es la edad promedio de los pasajeros del Titanic?

In [4]:
titanic["Age"].mean()

29.69911764705882

Hay diferentes estadísticas disponibles y se pueden aplicar a columnas con datos numéricos. Las operaciones en general excluyen los datos que faltan y operan en filas de forma predeterminada.

<img src="data/06_reduction.svg">

### 1.2 ¿Cuál es la edad media y el precio de la tarifa del billete de los pasajeros del Titanic?

In [5]:
titanic[["Age", "Fare"]].median()

Age     28.0000
Fare    14.4542
dtype: float64

*Observaciones*

La estadística aplicada a varias columnas de un DataFrame se calcula para cada columna numérica. La selección de dos columnas devuelve un DataFrame, consulte el tutorial de datos de subconjuntos.

La estadística de agregación se puede calcular para varias columnas al mismo tiempo. ¿Recuerdas la función `describe()` del primer tutorial?

In [7]:
titanic[["Age", "Fare"]].describe()

,Age,Fare
count,714.000000,891.000000
mean,29.699118,32.204208
std,14.526497,49.693429
min,0.420000,0.000000
25%,20.125000,7.910400
50%,28.000000,14.454200
75%,38.000000,31.000000
max,80.000000,512.329200


*Observaciones*

En lugar de las estadísticas predefinidas, se pueden definir combinaciones específicas de estadísticas agregadas para columnas dadas utilizando el  método `DataFrame.agg()`:

In [9]:
titanic.agg(
    {
        "Age": ["min", "max", "median", "skew"],
        "Fare": ["min", "max", "median", "mean"],
    }
)

,Age,Fare
min,0.420000,0.000000
max,80.000000,512.329200
median,28.000000,14.454200
skew,0.389108,NaN
mean,NaN,32.204208


## 2. Agregando estadísticas agrupadas por categoría 

<img src="data/06_groupby.svg">

### 2.1 ¿Cuál es la edad promedio de los pasajeros masculinos y femeninos del Titanic?

In [18]:
titanic[["Sex", "Age"]].groupby("Sex").mean()

,Age
Sex,
female,27.915709
male,30.726645


*Observaciones*

Como nuestro interés es la edad promedio para cada género, primero se hace una subselección en estas dos columnas. A continuación, se aplica el método para hacer un grupo por categoría. La edad promedio para cada sexo se calcula y se devuelve en un Dataframe.

Calcular una estadística dada (p. ej ., edad media) para cada categoría en una columna (p. ej., hombre/mujer en la columna `Sex`) es un patrón común. El método `groupby()` se utiliza para soportar este tipo de operaciones. Más general, esto encaja en el patrón `split-apply-combine` más general:

* Dividir los datos en grupos
* Aplicar una función a cada grupo de forma independiente
* Combinar los resultados en una estructura de datos

Los pasos de aplicar y combinar normalmente se realizan juntos en pandas.

En el ejemplo anterior, seleccionamos explícitamente las 2 columnas primero. Si no, el método `mean` se aplica a cada columna que contiene columnas numéricas:

In [19]:
titanic.groupby("Sex").mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


*Observaciones*

No tiene mucho sentido obtener el valor promedio de Pclass. si solo nos interesa la edad promedio para cada género, la selección de columnas (corchetes [] como de costumbre) también se admite en los datos agrupados:

In [21]:
titanic.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

<img src="data/06_groupby_select_detail.svg">

> Nota: La columna `Pclass` contiene datos numéricos pero en realidad representa 3 categorías (o factores) con las etiquetas '1', '2' y '3' respectivamente. Calcular estadísticas sobre estos no tiene mucho sentido. Por lo tanto, pandas proporciona un tipo de datos Categorico para manejar este tipo de datos. Se proporciona más información en la sección Datos categóricos de la guía del usuario.

### 2.2 ¿Cuál es el precio promedio de la tarifa del boleto para cada combinación de sexo y clase de cabina?

In [22]:
titanic.groupby(["Sex", "Pclass"])["Fare"].mean()

Sex     Pclass
female  1         106.125798
        2          21.970121
        3          16.118810
male    1          67.226127
        2          19.741782
        3          12.661633
Name: Fare, dtype: float64

La agrupación se puede hacer por varias columnas al mismo tiempo. Proporcione los nombres de las columnas como una lista para el método `groupby()`.

## 3. Contar número de registros por categoría

<img src="data/06_valuecounts.svg">

### 3.1 ¿Cuál es el número de pasajeros en cada una de las clases de cabina?

In [23]:
titanic["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

*Observaciones*

El método `value_counts()` cuenta el número de registros para cada categoría en una columna.

La función es un atajo, ya que en realidad es una operación groupby en combinación con el conteo del número de registros dentro de cada grupo:

In [24]:
titanic.groupby("Pclass")["Pclass"].count()

Pclass
1    216
2    184
3    491
Name: Pclass, dtype: int64

*Observaciones*

Ambos size y count se pueden utilizar en combinación con `groupby`. Mientras que size incluye valores NaN y solo proporciona el número de filas (tamaño de la tabla), countexcluye los valores que faltan. En el método `value_counts`, use el argumento dropna para incluir o excluir los valores NaN.

## RECORDAR
* Las estadísticas de agregación se pueden calcular en columnas o filas enteras

* groupby proporciona el poder del patrón `dividir-aplicar-combinar`

* value_counts es un atajo conveniente para contar el número de entradas en cada categoría de una variable